In [ ]:
# ====================== Match a date in the GDP dataframe with another date before that date in another dataframe ================================= #
# ======================================================================================================================== #

import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import datetime
import time

##### Set the current working directory
path="e:\\Copy\\SCRIPTS\\Forecast_Covid_Recovery\\Code\\"
os.chdir(path)

##### parse dates and times
def date_parser(date): 
    dt = datetime.datetime.strptime(date, '%m/%d/%Y')
    return dt.strftime('%Y-%m-%d') 

##### parse dates and times
def date_parser2(date): 
    dt = datetime.datetime.strptime(date, '%Y:%m:%d')
    return dt.strftime('%Y-%m-%d') 

##### Find the date which is 'n' days right after a date 'pivot' 
def closest_dates_after(date_list, pivot, n):
    key=lambda x: abs(x - pivot) 
    dates = [i for i in date_list if i > pivot]
    closest_dates = sorted(dates, key = key)[:n]
    found_dates = [str( closest_date.date() ) for closest_date in closest_dates] # get a list of string values of dates
    return found_dates[n-1] 

##### Find the date which is 'n' days before a date 'pivot' (if 'n' = 1, then the date found can be similar to 'pivot')
def closest_dates_before(date_list, pivot, n):
    key=lambda x: abs(x - pivot) 
    dates = [i for i in date_list if i <= pivot]
    closest_dates = sorted(dates, key = key)[:n]
    found_dates = [str( closest_date.date() ) for closest_date in closest_dates] # get a list of string values of dates
    return found_dates[n-1]

##### Combine two dataframes with dates of the second frame are 'n' periods on/before the dates of the first frame
def combine_df_before_dates(df1, df2, n):
    date_list1 = pd.to_datetime(df1['date'].tolist(), format='%Y-%m-%d')
    date_list2 = pd.to_datetime(df2['date'].tolist(), format='%Y-%m-%d')
    list_date1 = []
    list_date2 = []
    for date1 in date_list1:
        closest_date = closest_dates_before(date_list2, date1, n)
        list_date1.append( str( date1.date() ) )
        list_date2.append(closest_date)
    df1_new = df1.set_index('date').loc[list_date1, :].reset_index()
    df2_new = df2.set_index('date').loc[list_date2, :].reset_index()
    colnames2 = df2_new.columns.tolist()
    df2_new.set_axis([f'{colnames2[0]}_lag_{n}', f'{colnames2[1]}_lag_{n}'], axis = 1, inplace = True)
    combined_df = pd.concat([df1_new, df2_new], axis=1, join='inner')
    return combined_df

# import GDP data
colnames_GDP = ['date', 'GDP', 'OUTBS', 'OUTNFB', 'GPDIC1', 'INDPRO', 'CMRMTSPLx', 'IPMAT', 'FPIx', 'IPFINAL', 'IPDMAT']
fields = ['date', 'GDP', 'OUTBS', 'OUTNFB', 'GPDIC1', 'INDPRO', 'CMRMTSPLx', 'IPMAT', 'FPIx', 'IPFINAL', 'IPDMAT']
US_df = pd.read_csv('../Data\\FRED\\quarterly_transformed_pdc_sis.csv', engine = 'python', encoding='utf-8', \
                                                                                                                                                            skipinitialspace=True, usecols = fields, sep = ',', parse_dates=True)
US_df.set_axis(colnames_GDP, axis = 1, inplace = True)
US_df['date'] = US_df['date'].astype(str).apply(date_parser) # parse dates
US_df.dropna(inplace=True)
print( US_df.tail(10) )

# import WEI
fields = ['date', 'WEI']
colnames=["date", "WEI"]
WEI_df = pd.read_csv('../Data\\weekly-economic-index_data.csv', encoding='utf-8', skipinitialspace=True, usecols = fields, sep = ',', parse_dates = True)
WEI_df.set_axis(colnames, axis = 1, inplace = True)
WEI_df.dropna(inplace=True)
WEI_df['date'] = WEI_df['date'].astype(str).apply(date_parser) # parse dates
print( WEI_df.head(20) )

N = 6 # set maximum number of days before each release date of GDP
US1_df = US_df.loc[US_df['date'] >= WEI_df.loc[0, 'date'], :].copy()
print( US1_df.head() )
combined_df = reduce( lambda left, right: pd.merge(left, right, on = colnames_GDP), [combine_df_before_dates(US1_df, WEI_df, n) for n in np.arange(1, N)] )
col_removed = combined_df.columns.str.contains('date_lag')
combined_df = combined_df[combined_df.columns[col_removed == False] ].copy() # remove the selected columns
combined_df.to_csv('../Data/combined_data_WEI.csv', index=False, header = True)

# import ADS Index
fields_ADS = ['date', 'ADS_Index']
US_ADS_df = pd.read_csv('../Data\\ADS_Index_Most_Current_Vintage.csv', engine = 'python', encoding='utf-8', skipinitialspace=True, usecols = fields_ADS, sep = ',')
US_ADS_df.date = pd.to_datetime( US_ADS_df.date.astype(str).apply(date_parser2) )# parse dates
US_ADS_df.dropna(inplace=True)
US_ADS_df.set_index('date', inplace=True)
US_ADS_df_monthly = US_ADS_df.copy().resample('M').mean().reset_index() # convert a time series to the monthly frequencey by sampling then taking mean
print( US_ADS_df_monthly.head(20) )

N = 6 # set maximum number of days before each release date of GDP
US1_df = US_df.loc[pd.to_datetime(US_df['date']) >= US_ADS_df_monthly.loc[N-2, 'date'], :].copy()
print( US1_df.head() )
combined_df = reduce( lambda left, right: pd.merge(left, right, on = colnames_GDP), [combine_df_before_dates(US1_df, US_ADS_df_monthly, n) for n in np.arange(1, N)] )
col_removed = combined_df.columns.str.contains('date_lag')
combined_df = combined_df[combined_df.columns[col_removed == False] ].copy() # remove the selected columns
combined_df.to_csv('../Data/combined_data_ADSI.csv', index=False, header = True)

In [ ]:
# ==================================================== Transform FRED-QD data =================================================== #
# =========================================================================================================================== #
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import datetime
import time

##### Set the current working directory
path="e:\\Copy\\SCRIPTS\\Forecast_Covid_Recovery\\Code\\"
os.chdir(path)

##### parse dates and times
def date_parser(date): 
    dt = datetime.datetime.strptime(date, '%m/%d/%Y')
    return dt.strftime('%Y-%m-%d') 

##### Define data transformations
def transform_2(x: pd.Series):
    return x.diff(periods = 1)

def transform_3(x: pd.Series):
    x1 = transform_2(x)
    return x1.diff(periods = 1)

def transform_4(x: pd.Series):
    return np.log(x)

def transform_5(x: pd.Series):
    return np.log( x / x.shift(1) )

def transform_6(x: pd.Series):
    x1 = transform_5(x)
    return x1.diff(periods = 1)

def transform_7(x: pd.Series):
    x1 = x / x.shift(1) - 1.0
    return x1.diff(periods = 1)

US_df = pd.read_csv('../Data\\FRED\\quarterly1.csv', engine = 'python', encoding='utf-8', \
                                                                                                                                            skipinitialspace=True, sep = ',', parse_dates = ['date'], index_col = 'date')
US_df.fillna(US_df.mean(axis = 0), inplace = True)

# transform the data variables
step = 1
for i in list(np.arange(58, 64 + step, step) ) + list(np.arange(78, 79 + step, step) ) + list(np.arange(144, 151 + step, step) ) + list(np.arange(189, 193 + step, step) ) \
        + list(np.arange(197, 198 + step, step) ) + list(np.arange(201, 201 + step, step) ) + list(np.arange(220, 220 + step, step) ) + list(np.arange(223, 225 + step, step) ) \
        + list(np.arange(238, 238 + step, step) ) + list(np.arange(243, 243 + step, step) ) + list(np.arange(247, 247 + step, step) ):
    US_df.iloc[:, i-1] = transform_2( US_df.iloc[:, i-1] )

for i in list(np.arange(1, 10 + step, step) ) + list(np.arange(12, 12 + step, step) ) + list(np.arange(14, 32 + step, step) ) + list(np.arange(35, 57 + step, step) ) \
        + list(np.arange(65, 76 + step, step) ) + list(np.arange(81, 94 + step, step) ) + list(np.arange(128, 143 + step, step) ) + list(np.arange(158, 187 + step, step) ) \
        + list(np.arange(194, 196 + step, step) ) + list(np.arange(221, 222 + step, step) ) + list(np.arange(227, 232 + step, step) ) + list(np.arange(234, 234 + step, step) ) \
        + list(np.arange(236, 237 + step, step) ) + list(np.arange(239, 239 + step, step) ) + list(np.arange(241, 242 + step, step) ) + list(np.arange(244, 246 + step, step) ) \
        + list(np.arange(248, 248 + step, step) ):
    US_df.iloc[:, i-1] = transform_5( US_df.iloc[:, i-1] )

for i in list(np.arange(95, 127 + step, step) ) + list(np.arange(199, 199 + step, step) ) + list(np.arange(205, 219 + step, step) ) + list(np.arange(233, 233 + step, step) ):
    US_df.iloc[:, i-1] = transform_6( US_df.iloc[:, i-1] )

for i in list(np.arange(200, 200 + step, step) ):
    US_df.iloc[:, i-1] = transform_7( US_df.iloc[:, i-1] )

US_df.to_csv('../Data/FRED/quarterly_transformed.csv', index=True, header = True)
# US_df.plot(subplots=False, color='blue', figsize=(10, 8), grid=True, rot=10, sharex=True, sharey=False)
# US_df.dropna(inplace=True)
 # drop variables with many missing observations
US_df_PCA = US_df.drop(['OUTMS', 'TCU', 'LNS13023621',	'LNS13023557', 'LNS13023705', 'LNS13023569', 'HOAMS', \
    'ACOGNOx', 'ANDENOx', 'INVCQRMTSPL', 'COMPRMS', 'OPHMFG', 'ULCMFG', 'IMFSLx', 'DRIWCIL', 'USSTHPI', \
    'SPCS10RSA', 'SPCS20RSA', 'TWEXAFEGSMTHx',	'EXUSEU', 'USEPUINDXM', 'MORTG10YRx', 'MORTGAGE30US'], axis = 1)
US_df_PCA.dropna(inplace=True)
US_df_PCA.to_csv('../Data/FRED/quarterly_transformed_pca.csv', index=True, header = True)


In [2]:
# ================================ ML with H2O: Calculate in-sample and out-of-sample MAD, MAE, RMSE, R^2 ================================= #
# =========================================================================================================================== #
import pandas as pd
import numpy as np
import os
import sys
from varname import nameof
import math
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels import robust
from statsmodels.tsa.api import VAR
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# import H2O
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2OXGBoostEstimator # (not supported on Windows OS)
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

# import a Lasso module for time series analysis
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

# import Random Forest module
from sklearn.ensemble import RandomForestRegressor
# import XGBoost module
from xgboost import XGBRegressor

# import Prophet
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.utilities import regressor_coefficients 
import utils_fprophet

# import tensorflow
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, LSTM, Dropout, SimpleRNN
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.layers import Flatten, Conv1D, MaxPooling1D

import warnings
import datetime
import time

import multiprocessing as multi

##### Set the current working directory
path="e:\\Copy\\SCRIPTS\\Forecast_Covid_Recovery\\Code\\"
os.chdir(path)

path_midas = "./midas/"
sys.path.append(path_midas)

from mix import mix_freq, mix_freq2
from adl import estimate, forecast, midas_adl, rmse, estimate2,forecast2, midas_adl2


# os.environ['NUMEXPR_MAX_THREADS'] = '30'


##### Call H2O algorithms to perform out-of-sample forecast
def H2Of(R: np.array, X: np.array, tau: int, use_model = 'randomforest', max_models = 3, max_runtime_secs = 600, max_runtime_secs_per_model = 40) -> tuple:
    """
        R: a numpy vector of responses
        X: a numpy array of predictors
        tau: a forecast horizon 
        max_models: a maximum number of models to be trained
        max_runtime_secs: a maximum runtime
        max_runtime_secs_per_model: a maximum runtime to train each individual model
    """
    assert (R.shape[0] == X.shape[0]), "numbers of rows not match!"
    assert(tau > 0), "the forecast horizon must be greater than zero!"
    
    T = X.shape[0]
    dim = X.shape[1]
    # R = R.flatten()
    # X = X.flatten() # flatten arrays

    R1 = np.empty( shape = (0, 1) )
    X1 = np.empty( shape = (0, dim) )
    for t in np.arange(0, T):
        if t < T-tau:
            R1 = np.append(R1, R[t+tau].reshape(1, 1), axis = 0)
        else:
            R1 = np.append(R1, np.array([0]).reshape(1, 1), axis = 0)
        X1 = np.append(X1, X[t, :].reshape(1, dim), axis = 0)

    # preprocess data
    if use_model == 'deeplearning':
        scaler = MinMaxScaler(feature_range=(0, 1))
    else:
        scaler = StandardScaler()
    X1 = scaler.fit_transform(X1)

    # split the numpy array into train, validation, and test data
    sratio = 0.8
    X1_train, X1_valid, X1_test = X1[0:(math.floor(sratio*T)-tau), :].reshape(-1, dim), X1[(math.floor(sratio*T)-tau):(T-tau), :].reshape(-1, dim), \
                                                                                                                                                                                X1[(T-tau):, :].reshape(-1, dim)
    R1_train, R1_valid, R1_test = R1[0:(math.floor(sratio*T)-tau), :].reshape(-1, 1), R1[(math.floor(sratio*T)-tau):(T-tau), :].reshape(-1, 1), \
                                                                                                                                                                                    R1[(T-tau):, :].reshape(-1, 1)

    # put all data into H2O dataframes
    train_hf = h2o.H2OFrame(python_obj=np.concatenate( (R1_train, X1_train), axis = 1 ), column_names=['x'+str(i) for i in range(dim+1)])
    train_hf.rename(columns = {'x0': 'y'}) 
    valid_hf = h2o.H2OFrame(python_obj=np.concatenate( (R1_valid, X1_valid), axis = 1 ), column_names=['x'+str(i) for i in range(dim+1)])
    valid_hf.rename(columns = {'x0': 'y'}) 
    test_hf = h2o.H2OFrame(python_obj=np.concatenate( (R1_test, X1_test), axis = 1 ), column_names=['x'+str(i) for i in range(dim+1)])
    test_hf.rename(columns = {'x0': 'y'}) 

    # build and train the model
    if use_model == 'randomforest':
        model = H2OAutoML(project_name = "AutoML_RF", max_models = max_models, max_runtime_secs = max_runtime_secs, nfolds = 0, \
                                                max_runtime_secs_per_model = max_runtime_secs_per_model, include_algos = ["DRF"], keep_cross_validation_predictions = True, \
                                                stopping_metric = 'RMSE', sort_metric = 'RMSE', seed = 1234, verbosity = 'warn')
    elif use_model == 'gbm':
        model = H2OAutoML(project_name = "AutoML_GBM", max_models = max_models, max_runtime_secs = max_runtime_secs, nfolds = 0, \
                                                max_runtime_secs_per_model = max_runtime_secs_per_model, include_algos = ["GBM"], keep_cross_validation_predictions = True, \
                                                stopping_metric = 'RMSE', sort_metric = 'RMSE', seed = 1234, verbosity = 'warn')
    elif use_model == 'xgboost':
        model = H2OAutoML(project_name = "AutoML_XGB", max_models = max_models, max_runtime_secs = max_runtime_secs, nfolds = 0, \
                                                max_runtime_secs_per_model = max_runtime_secs_per_model, include_algos = ["XGBoost"], keep_cross_validation_predictions = True, \
                                                stopping_metric = 'RMSE', sort_metric = 'RMSE', seed = 1234, verbosity = 'warn')
    elif use_model == 'deeplearning':
        model = H2OAutoML(project_name = "AutoML_DL", max_models = max_models, max_runtime_secs = max_runtime_secs, nfolds = 0, \
                                                max_runtime_secs_per_model = max_runtime_secs_per_model, include_algos = ["DeepLearning"], keep_cross_validation_predictions = True, \
                                                stopping_metric = 'RMSE', sort_metric = 'RMSE', seed = 1234, verbosity = 'warn')
    elif use_model == 'glm':
        model = H2OAutoML(project_name = "AutoML_GLM", max_models = max_models, max_runtime_secs = max_runtime_secs, nfolds = 0, \
                                                max_runtime_secs_per_model = max_runtime_secs_per_model, include_algos = ["GLM"], keep_cross_validation_predictions = True, \
                                                stopping_metric = 'RMSE', sort_metric = 'RMSE', seed = 1234, verbosity = 'warn')
    else:
        print(f'Model {use_model} does not exist!')
        sys.exit()
    model.train(x=list(set(train_hf.names)-{"y"}), y="y", training_frame=train_hf, validation_frame=valid_hf)

    # get optimal hyperparameters
    best_model = model.get_best_model(criterion='RMSE')
    opt_params = best_model.summary().as_data_frame().iloc[:, 1:]
    opt_params = opt_params.to_dict('records')[0]
 
    # get performance measures for the validation data
    best_model_perf = best_model.model_performance(valid_hf)
    rmse = best_model_perf.rmse()
    mae = best_model_perf.mae()

    # forecast for the test data
    forecasts = model.predict(test_hf).as_data_frame(use_pandas=True, header=True).values
    # print('forecasts = \n', forecasts)
    forecast_tau = float(forecasts[len(forecasts) - 1])

    # delete all the H2O data and model
    h2o.remove(model)
    h2o.remove(best_model)
    h2o.remove(train_hf)
    h2o.remove(valid_hf)
    h2o.remove(test_hf)

    # output results
    return forecast_tau, rmse, mae, opt_params

##### Calculate rolling-window OoS R^2 for 'tau'-steps ahead forecasts
def OoS_R_sq(df: pd.DataFrame, tau: int, wsize: int, sstart:int, T1: int, ylag: int, use_model = 'randomforest', max_models = 3, \
                                                                                                                                    max_runtime_secs = 600, max_runtime_secs_per_model = 40) -> tuple:
    """ 
        'df': a pandas dataframe
        'tau': forecast horizon
        'wsize': window size,
        'sstart': sub-sample starting point
        'T1': size of a sub-sample
        'ylag': AR lag of the dependent variable 
        'max_models': a maximum number of models to be trained
        'max_runtime_secs': a maximum runtime 
        'max_runtime_secs_per_model': a maximum runtime to train each individual model
    """
    assert (T1 > wsize+tau), "size of a subsample must be much greater than the window size!"
    assert ( sstart+T1 <= len(df) ), "end point of the last subsample must be less than or equal to size of the dataframe!"

    df = df.iloc[sstart:(sstart+T1), :].copy()
    # get data for the dependant variable and predictors
    if ylag > 0:
        for i in np.arange(1, ylag+1):
            df[f'ylag{i}'] = df.iloc[:, 1].shift(i)
        df.dropna(inplace = True)
    # print( df.iloc[0, 0] )

    R = np.array(df.values[:, 1], dtype='float64')
    X = np.array(df.values[:, 2:], dtype='float64')
    dim = X.shape[1]

    rmse = 0
    var = 0
    mae = 0
    list_err = []
    rmse_in_ls, mae_in_ls = [], []
    list_opt_params_vlues = []
    for s in np.arange(T1-wsize-tau-ylag):
        # print('iter: ', s)
        # estimate a long-run regression model and make a 'tau'-steps ahead forecast
        if use_model == use_model:
            r_forecast, rmse_in, mae_in, opt_params = H2Of(R[s:(s+wsize+1)].reshape(-1, 1), X[s:(s+wsize+1), :].reshape(-1, dim), tau, \
                                                                                            use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                                            max_runtime_secs_per_model = max_runtime_secs_per_model)
            list_opt_params_keys = [k for k in opt_params.keys()]
            list_opt_params_vlues.append( list( opt_params.values() ) )
            rmse_in_ls.append(rmse_in)
            mae_in_ls.append(mae_in)
        else:
            print(f'Model {use_model} does not exist!')
            sys.exit()

        r = R[s+wsize+tau] # actual returns
        rmse +=  pow(r - r_forecast, 2) / (T1-wsize-tau-ylag)
        var += pow(r - np.mean(R[s:(s+wsize+1)]), 2) / (T1-wsize-tau-ylag)
        mae += abs(r - r_forecast) / (T1-wsize-tau-ylag)
        list_err.append(r - r_forecast)
    err = np.array(list_err)
    mad = robust.mad(err, c = 1)

    # save optimal hyperparameters to a dataframe
    list_opt_params_vlues = np.array(list_opt_params_vlues)
    opt_params_df = pd.DataFrame({list_opt_params_keys[i]: list_opt_params_vlues[:,i] for i in np.arange( len(list_opt_params_keys) )})
    
    del df # delete this copy of the dataframe
    return float(mad), float(mae), math.sqrt(float(rmse) ), float(1 - rmse/(var + 0.00001) ), np.median(rmse_in_ls), np.median(mae_in_ls), \
                                                                                                                                                    opt_params_df.select_dtypes(include=np.number).median(axis=0)

##### Calculate values of the rolling-window OoS R^2 of 'tau'-steps ahead forecasts for many different moving-forward sub-samples
def OoS_R_sq_many_ssamples(df: pd.DataFrame, df_name: 'string', tau: int, step_size, wsize: int, T1, ylag, use_model = 'deeplearning', \
                                                    max_models = 100, max_runtime_secs = 600, max_runtime_secs_per_model = 40) -> tuple:
    ssize = len(df) - ylag                                                                                                                                                                                                        
    print(f'Entire sample size = {ssize}; Sub-sample size = {T1}; rolling-window size = {wsize}; AR lag = {ylag}; forecast horizon = {tau}; model = {use_model}')
    perf_dict = {'sstart': [], 'ssample_start_date': [], 'ssample_end_date': [], 'mad': [], 'mae': [], 'rmse': [], 'r_sq': [], 'rmse_in_med': [], 'mae_in_med': []}
    opt_params_med_ls = []
    for sstart in np.arange(0, ssize+step_size, step_size):
        if sstart+T1 <= ssize:
            mad, mae, rmse, r_sq, rmse_in_med, mae_in_med, opt_params_med = OoS_R_sq(df, tau, wsize, sstart, T1, ylag, use_model = use_model, \
                                                                                                                                                    max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                                                                                                    max_runtime_secs_per_model = max_runtime_secs_per_model)
            print(f'MAD = {mad:.4f}, MAE = {mae:.4f}, RMSE = {rmse:.4f}, OoS R^2 = {r_sq:.4f}')
            perf_dict['sstart'].append(sstart)
            perf_dict['ssample_start_date'].append(df.iloc[sstart, 0])
            perf_dict['ssample_end_date'].append(df.iloc[sstart+T1-1, 0])
            perf_dict['mad'].append(f'{mad:.4f}')
            perf_dict['mae'].append(f'{mae:.4f}')
            perf_dict['rmse'].append(f'{rmse:.4f}')
            perf_dict['r_sq'].append(f'{r_sq:.4f}')
            perf_dict['rmse_in_med'].append(f'{rmse_in_med:.4f}')
            perf_dict['mae_in_med'].append(f'{mae_in_med:.4f}')
            opt_params_med_columns = list(opt_params_med.index)
            opt_params_med_ls.append( list(opt_params_med.values) )
    # save performance metrics to a data frame
    perf_df = pd.DataFrame.from_dict(perf_dict)
    perf_df.to_csv(f'../Data/FRED/perf_out_{df_name}_model_{use_model}_ssize_{ssize}_subsize_{T1}_fhorizon_{tau}_ylag_{ylag}.csv', index=True, header = True)

    # save medians of optimal hyperparameters to a data frame
    opt_params_med_ls = np.array(opt_params_med_ls)
    opt_params_med_df = pd.DataFrame({opt_params_med_columns[i]: opt_params_med_ls[:,i] for i in np.arange( len(opt_params_med_columns) )})
    opt_params_med_df.insert(loc = 0, column = 'sstart', value = perf_df.sstart)
    opt_params_med_df.insert(loc = 1, column = 'ssample_start_date', value = perf_df.ssample_start_date)
    opt_params_med_df.insert(loc = 2, column = 'ssample_end_date', value = perf_df.ssample_end_date)
    opt_params_med_df.to_csv(f'../Data/FRED/opt_params_out_{df_name}_model_{use_model}_ssize_{ssize}_subsize_{T1}_fhorizon_{tau}_ylag_{ylag}.csv', index=True, header = True)
    return perf_df, opt_params_med_df


if __name__ == '__main__':
    start_time = time.time()

    ##### Import data
    # import a sample with many predictors
    US_df_big = pd.read_csv('../Data/FRED/quarterly_transformed_pca.csv', engine = 'python', encoding='utf-8', skipinitialspace=True, sep = ',', parse_dates=True)
    display(US_df_big.head())

    # import a sample with a few predictors
    US_df_small = pd.read_csv('../Data/FRED/quarterly_transformed_pdc_sis.csv', engine = 'python', encoding='utf-8', \
                                                                                                                       skipinitialspace=True, sep = ',', parse_dates=True)
    
	# import ADSI in addition to FRED variables selected by variable screening 
    US_df_ADS_small = pd.read_csv('../Data/combined_data_ADSI.csv', engine = 'python', encoding='utf-8', skipinitialspace=True, sep = ',', parse_dates=True)
                                                                                                                                                                                    

    ##### Launch a H2O instance
    h2o.init(ip="localhost", port=54323, max_mem_size_GB=100, nthreads=-1) 
    # h2o.cluster().shutdown()
    h2o.no_progress() # suppress progress bars


    batch_size = 40
    num_epochs = 100
    wsize = 60
    ylag = 1
    ssize = len(US_df_big) - ylag
    step = 1
    T1 = 100
    max_models = 10
    max_runtime_secs = 0
    max_runtime_secs_per_model = 20

     ####################################################### Forecast with 'US_df_big' #######################################################################
    tau_step = 1
    taus = np.arange(1, 4+tau_step, tau_step)
    for tau in taus:
        use_model = 'deeplearning'
        perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_big, 'US_df_big', tau, step, wsize, T1, ylag, \
                                                                use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                max_runtime_secs_per_model = max_runtime_secs_per_model)
        print( perf_df.head() )
        print( opt_params_med_df.head() )

        use_model = 'randomforest'
        perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_big, nameof(US_df_big), tau, step, wsize, T1, ylag, \
                                                                use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                max_runtime_secs_per_model = max_runtime_secs_per_model)
        print( perf_df.head() )
        print( opt_params_med_df.head() )

        use_model = 'gbm'
        perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_big, 'US_df_big', tau, step, wsize, T1, ylag, \
                                                                use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                max_runtime_secs_per_model = max_runtime_secs_per_model)
        print( perf_df.head() )
        print( opt_params_med_df.head() )

        use_model = 'xgboost'
        perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_big, 'US_df_big', tau, step, wsize, T1, ylag, \
                                                                use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                max_runtime_secs_per_model = max_runtime_secs_per_model)
        print( perf_df.head() )
        print( opt_params_med_df.head() )

    ####################################################### Forecast with 'US_df_small' ###########################################################################

    tau_step = 1
    taus = np.arange(1, 4+tau_step, tau_step)
    for tau in taus:
        if tau > 2:
            use_model = 'randomforest'
            perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_small, nameof(US_df_small), tau, step, wsize, T1, ylag, \
                                                                    use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                    max_runtime_secs_per_model = max_runtime_secs_per_model)
            print( perf_df.head() )
            print( opt_params_med_df.head() )

            use_model = 'gbm'
            perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_small, nameof(US_df_small), tau, step, wsize, T1, ylag, \
                                                                    use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                    max_runtime_secs_per_model = max_runtime_secs_per_model)
            print( perf_df.head() )
            print( opt_params_med_df.head() )

            use_model = 'deeplearning'
            perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_small, nameof(US_df_small), tau, step, wsize, T1, ylag, \
                                                                    use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                    max_runtime_secs_per_model = max_runtime_secs_per_model)
            print( perf_df.head() )
            print( opt_params_med_df.head() )

            use_model = 'xgboost'
            perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_small, nameof(US_df_small), tau, step, wsize, T1, ylag, \
                                                                    use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                    max_runtime_secs_per_model = max_runtime_secs_per_model)
            print( perf_df.head() )
            print( opt_params_med_df.head() )

            use_model = 'glm'
            perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_small, nameof(US_df_small), tau, step, wsize, T1, ylag, \
                                                                    use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                    max_runtime_secs_per_model = max_runtime_secs_per_model)
            print( perf_df.head() )
            print( opt_params_med_df.head() )



    ####################################################### Forecast with 'US_df_ADS_small' #######################################################################

    tau_step = 1
    taus = np.arange(1, 4+tau_step, tau_step)
    for tau in taus:
        if tau > 1:
            use_model = 'randomforest'
            perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_ADS_small, nameof(US_df_ADS_small), tau, step, wsize, T1, ylag, \
                                                                    use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                    max_runtime_secs_per_model = max_runtime_secs_per_model)
            print( perf_df.head() )
            print( opt_params_med_df.head() )

            use_model = 'gbm'
            perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_ADS_small, nameof(US_df_ADS_small), tau, step, wsize, T1, ylag, \
                                                                    use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                    max_runtime_secs_per_model = max_runtime_secs_per_model)
            print( perf_df.head() )
            print( opt_params_med_df.head() )

        use_model = 'deeplearning'
        perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_ADS_small, nameof(US_df_ADS_small), tau, step, wsize, T1, ylag, \
                                                                use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                max_runtime_secs_per_model = max_runtime_secs_per_model)
        print( perf_df.head() )
        print( opt_params_med_df.head() )

        use_model = 'xgboost'
        perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_ADS_small, nameof(US_df_ADS_small), tau, step, wsize, T1, ylag, \
                                                                use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
                                                                max_runtime_secs_per_model = max_runtime_secs_per_model)
        print( perf_df.head() )
        print( opt_params_med_df.head() )

        # use_model = 'glm'
        # perf_df, opt_params_med_df = OoS_R_sq_many_ssamples(US_df_ADS_small, nameof(US_df_ADS_small), tau, step, wsize, T1, ylag, \
        #                                                         use_model = use_model, max_models = max_models, max_runtime_secs = max_runtime_secs, \
        #                                                         max_runtime_secs_per_model = max_runtime_secs_per_model)
        # print( perf_df.head() )
        # print( opt_params_med_df.head() )





    ##### Close the H2O instance
    h2o.cluster().shutdown()

    print( 'Completed in: %s sec'%(time.time() - start_time) )

c:\Users\BaChu\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'e:\\Copy\\SCRIPTS\\Forecast Covid Recovery\\Code\\'